In [1]:
from dotenv import load_dotenv
load_dotenv("../.env")

True

In [2]:
from pprint import pprint

from datasets import load_dataset
import tqdm

from graph import create_debug_agent_graph
from metrics.pass_at_k import estimate_pass_at_k

/Users/romanavanesov/miniconda3/envs/LangGraph-experiments/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset("bigcode/humanevalpack", "python")
problems = list(dataset["test"])

#take only first 25 problems for quicker testing
problems = problems[:2]

print("problems len: " , len(problems))
print(problems[0].keys())

problems len:  2
dict_keys(['task_id', 'prompt', 'declaration', 'canonical_solution', 'buggy_solution', 'bug_type', 'failure_symptoms', 'entry_point', 'import', 'test_setup', 'test', 'example_test', 'signature', 'docstring', 'instruction'])


In [4]:
experiment_problem = problems[0]
print("Problem ID:", experiment_problem["task_id"])
print("Problem prompt: ", experiment_problem["prompt"])
print("Problem declaration: ", experiment_problem["declaration"])
print("Problem canonical_solution: ", experiment_problem["canonical_solution"])
print("Problem buggy_solution: ", experiment_problem["buggy_solution"])
print("Problem instruction: ", experiment_problem["instruction"])

Problem ID: Python/0
Problem prompt:  from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

Problem declaration:  from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:

Problem canonical_solution:      for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = abs(elem - elem2)
                if distance < threshold:
                    return True

    return False

Problem buggy_solution:      for idx, elem in enumerate(numbers):
        for idx2, elem2 in enumerate(numbers):
            if idx != idx2:
                distance = elem - elem2
                if distance <

In [5]:
graph = create_debug_agent_graph()

In [6]:
from agent.state import DebugAgentState
from langchain_core.messages import HumanMessage

from typing import Any, Dict

def fix_code(buggy_code, test_code="", max_iterations=5) -> Dict[str, Any]:
    """
    Uses the debug agent graph to attempt to fix the provided buggy code.
    """

    user_message = f"Fix the following Python code:\n{buggy_code}"

    initial_state: DebugAgentState = {
        "messages": [HumanMessage(content=user_message)],
        "iterations": 0,
        "max_iterations": max_iterations,
        "original_buggy_code": buggy_code,
        "test_code": test_code,
        "fixed_code": "",
        "is_fixed": False,
        "submit_idx": -1,
        "submissions": [],
        "first_pass": None
    }

    final_state = graph.invoke(initial_state)

    return {
        "fixed_code": final_state["fixed_code"],
        "is_fixed": final_state["is_fixed"],
        "iterations": final_state["iterations"],
        "messages": final_state["messages"],
        "submissions": final_state["submissions"],
        "first_pass": final_state["first_pass"],
    }


In [7]:
results = []

for problem in tqdm.tqdm(problems):
    buggy_solution = problem.get("buggy_solution", "")
    test_code = problem.get("test_code", "")
    results.append(fix_code(buggy_solution, test_code=test_code))

100%|██████████| 2/2 [01:38<00:00, 49.17s/it]


In [8]:
from pprint import pprint
pprint(results)

[{'first_pass': None,
  'fixed_code': 'def check_threshold(numbers, threshold):\n'
                '    for idx, elem in enumerate(numbers):\n'
                '        for idx2, elem2 in enumerate(numbers):\n'
                '            if idx != idx2:\n'
                '                distance = abs(elem - elem2)\n'
                '                if distance < threshold:\n'
                '                    return True\n'
                '\n'
                '    return False',
  'is_fixed': False,
  'iterations': 5,
  'messages': [HumanMessage(content='Fix the following Python code:\n    for idx, elem in enumerate(numbers):\n        for idx2, elem2 in enumerate(numbers):\n            if idx != idx2:\n                distance = elem - elem2\n                if distance < threshold:\n                    return True\n\n    return False\n', additional_kwargs={}, response_metadata={}, id='29ab0942-83e6-40a0-94d4-b911419dda39'),
               AIMessage(content="The provided code